# SQLAlchemy example


In [8]:
import pandas as pd
#sqlalchemy is a dependency for pd.to_sql
from sqlalchemy import create_engine, Column, Integer, String, Boolean, inspect
from sqlalchemy.orm import declarative_base, sessionmaker

In [3]:
#database set up
database_file = 'places.db'
engine = create_engine('sqlite:///'+database_file)
Base = declarative_base()

In [4]:
#input data
data = {
    'id': [1, 2, 3, 4],
    'place_name': ['Parthenon', 'Jupiter', 'Lake Nemi', 'Athens'],
    'place_address': ['123 Main St', '456 Oak Ave', '789 Elm Rd', '101 Pine Blvd'],
    'protected': [True, False, True, False]
}

df = pd.DataFrame(data)

In [5]:
class Places(Base):
    __tablename__ = 'places'
    #to get the string lengths, you have to look at the data for the max lengths of the input data
    id = Column(Integer, primary_key=True)
    place_name = Column(String(9), unique=True, nullable=False)
    place_address  = Column(String(13), unique=True, nullable=False)
    protected  = Column(Boolean,  nullable=True)
#The following line creates table if it doesn't exist,
#   and needs to be after you define your model (your table), so the metadata is properly saved
Base.metadata.create_all(engine)

#sqlalchemy orm uses sessions
Session = sessionmaker(bind=engine)
session = Session()

## Note about pandas.to_sql and sqlite3
The documentation tells you that pandas.to_sql will not maintain primary key integrity. sqlite3 doesn't allow you to use alter table to add a primary key constraint either. If you use to_sql and sqlite3 together, this is an issue. You can still read and update the table, but you won't have integrity constraints that you set up in the orm model.

In [6]:
#this adds the data in with one line
df.to_sql('places', con=engine, if_exists='append', index=False)

4

In [9]:
print("primary key is id :", inspect(Places).primary_key[0].name)
inspector = inspect(engine)
existing_tables = inspector.get_table_names()
print("this says places table exists :", existing_tables)
print("table is also in metadata :", Base.metadata.tables.keys())
#if Base.metadata.create_all(engine) isn't after your class for the data,
#   inspect can see your table, but the metadata can't.

primary key is id : id
this says places table exists : ['places']
table is also in metadata : dict_keys(['places'])


In [10]:
all_places = session.query(Places).all()
print()
print("all places:")
for p in all_places:
    print(p.id, p.place_name, p.place_address, p.protected)


all places:
1 Parthenon 123 Main St True
2 Jupiter 456 Oak Ave False
3 Lake Nemi 789 Elm Rd True
4 Athens 101 Pine Blvd False


In [11]:
places = session.query(Places).filter_by(place_name='Parthenon').first()
print(places.id, places.place_name, places.place_address, places.protected)

1 Parthenon 123 Main St True


In [12]:
session.close()